<a href="https://colab.research.google.com/github/andrewdk1123/KoSentiment/blob/main/Simple_RNN_for_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

이 Notebook에서는 PyTorch를 이용하여 주어진 문장의 감정 (label 0: neg, 1: pos)을 분류하는 간단한 RNN 모델을 만들어 보겠습니다.

RNN 모델은 입력 시퀀스에 대해 순차적으로 Hidden State를 계산하는 모델입니다. 즉, 현재 Timestep의 Hidden State를 계산할 때, 이전 Timestep에서 계산된 Hidden State를 함께 계산함으로써, 입력 시퀀스의 순서를 고려하여 학습할 수 있습니다.

모델은 다음과 같이 구성됩니다.

 * Embedding Layer: 입력 문장을 단어 임베딩으로 변환합니다.
 * RNN Layer: 입력 시퀀스에 대해 순차적으로 Hidden State를 계산합니다.
 * Linear Layer: 마지막 Hidden State를 입력으로 받아 감정을 분류합니다.

# Data Preparation

In [1]:
# Load packages
import pandas as pd
import numpy as np
import io

## Upload Train and Test Set

In [2]:
from google.colab import files

uploaded = files.upload()

Saving processed_training.csv to processed_training.csv


In [3]:
print(uploaded.keys())

dict_keys(['processed_training.csv'])


In [4]:
train_data = pd.read_csv(io.BytesIO(uploaded['processed_training.csv']), sep = ',')

cols_to_keep = ['label', 'sentence', 'tokenized_sentence', 'cleaned_tokens']
train_data = train_data.loc[:, cols_to_keep]
train_data.head()

,label,sentence,tokenized_sentence,cleaned_tokens
0,0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,"['일은', '왜', '해도', '해도', '끝이', '없을까', '?', '화가'...","['일은', '왜', '해도', '해도', '끝이', '없을까', '화가', '난다']"
1,0,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,"['이번', '달에', '또', '급여', '##가', '깎', '##였어', '!...","['이번', '달에', '또', '급여', '깎', '물가', '오르는', '월급'..."
2,0,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,"['회사에', '신입', '##이', '들어왔', '##는데', '말투', '##가...","['회사에', '신입', '들어왔', '말투', '거슬', '그런', '애를', '..."
3,0,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,"['직장', '##에서', '막내', '##라는', '이유로', '나에게', '##...","['직장', '막내', '이유로', '나에게', '온갖', '심', '시켜', '일..."
4,0,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,"['얼마', '전', '입사', '##한', '신입', '##사원', '##이', ...","['얼마', '전', '입사', '신입', '나를', '무시', '것', '같아서'..."


In [8]:
uploaded = files.upload()

Saving processed_test.csv to processed_test (1).csv


In [9]:
print(uploaded.keys())

dict_keys(['processed_test (1).csv'])


In [11]:
test_data = pd.read_csv(io.BytesIO(uploaded['processed_test (1).csv']), sep = ',')

test_data = test_data.loc[:, cols_to_keep]
test_data.head()

,label,sentence,tokenized_sentence,cleaned_tokens
0,0,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...,"['이번', '프로젝트', '##에서', '발표를', '하는데', '내가', '실수...","['이번', '프로젝트', '발표를', '하는데', '내가', '실수', '바람에'..."
1,0,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.,"['회사에서', '중요한', '프로젝트를', '혼자', '하게', '됐는데', '솔...","['회사에서', '중요한', '프로젝트를', '혼자', '하게', '됐는데', '솔..."
2,0,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.,"['상', '##사가', '너무', '무섭게', '생겨서', '친', '##해지는'...","['상', '너무', '무섭게', '생겨서', '친', '게', '너무', '두려워']"
3,0,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.,"['이번에', '힘들게', '들어간', '첫', '직장', '##이거', '##든'...","['이번에', '힘들게', '들어간', '첫', '직장', '첫', '직장', '그..."
4,0,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.,"['직장', '##에서', '동료', '##들이랑', '관계가', '안', '좋아질...","['직장', '동료', '관계가', '안', '좋아질', '봐', '걱정']"


## Data Preparation

In [53]:
import torch
from torch.nn.utils.rnn import pad_sequence

In [47]:
import ast

# Convert the string representation of the list to an actual list
train_data['cleaned_tokens'] = train_data['cleaned_tokens'].apply(ast.literal_eval)
test_data['cleaned_tokens'] = test_data['cleaned_tokens'].apply(ast.literal_eval)

In [54]:
# Flatten the list of tokens
all_tokens = train_data['cleaned_tokens'].explode()

# Build a vocabulary using Counter
vocab = Counter(all_tokens)

# Create word-to-index and index-to-word dictionaries
word_to_index = {word: index + 1 for index, (word, _) in enumerate(vocab.most_common())}
index_to_word = {index: word for word, index in word_to_index.items()}

# Add a special token for padding with index 0
word_to_index['<PAD>'] = 0
index_to_word[0] = '<PAD>'

# Encode tokens using the word-to-index dictionary
train_data['encoded_tokens'] = train_data['cleaned_tokens'].apply(lambda tokens: [word_to_index[token] for token in tokens])

# Pad sequences to a specific length (adjust maxlen as needed)
maxlen = 20
padded_tokens = pad_sequence([torch.tensor(tokens) for tokens in train_data['encoded_tokens']], batch_first=True, padding_value=word_to_index['<PAD>']).tolist()
train_data['padded_tokens'] = [list(tokens) for tokens in padded_tokens]

In [57]:
train_data.iloc[:,-2:].head()

,encoded_tokens,padded_tokens
0,"[684, 34, 322, 322, 2465, 2290, 54, 1641]","[684, 34, 322, 322, 2465, 2290, 54, 1641, 0, 0..."
1,"[121, 1328, 64, 2434, 4978, 4979, 4980, 525, 6...","[121, 1328, 64, 2434, 4978, 4979, 4980, 525, 6..."
2,"[255, 739, 1434, 3578, 4586, 109, 1657, 209, 2...","[255, 739, 1434, 3578, 4586, 109, 1657, 209, 2..."
3,"[42, 1834, 823, 137, 3460, 618, 1482, 564, 252...","[42, 1834, 823, 137, 3460, 618, 1482, 564, 252..."
4,"[148, 196, 645, 739, 18, 294, 6, 71, 1, 54, 4]","[148, 196, 645, 739, 18, 294, 6, 71, 1, 54, 4,..."


In [58]:
print("Length of the Vocabulary is: ", len(vocab))

Length of the Vocabulary is:  12989


In [60]:
# Example: Reconstruct tokens for the first row
example_row_index = 0
reconstructed_tokens = [index_to_word[index] for index in train_data.loc[example_row_index, 'padded_tokens'] if index != 0]

# Display the reconstructed tokens
print(reconstructed_tokens)

['일은', '왜', '해도', '해도', '끝이', '없을까', '화가', '난다']
